In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import datetime, time, gc

from tqdm import tqdm

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

In [2]:
offline_dataframe = pd.read_csv('ccf_offline_stage1_train.csv')
# online_dataframe = pd.read_csv('ccf_online_stage1_train.csv')
test_dataframe = pd.read_csv('ccf_offline_stage1_test_revised.csv')

In [3]:
offline_dataframe.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [4]:
test_dataframe.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [5]:
coupon_not_used = offline_dataframe[(offline_dataframe['Coupon_id'] != 'null') & (offline_dataframe['Date'] == 'null')]
coupon_used = offline_dataframe[(offline_dataframe['Coupon_id'] != 'null') & (offline_dataframe['Date'] != 'null')]
no_coupon = offline_dataframe[(offline_dataframe['Coupon_id'] == 'null') & (offline_dataframe['Date'] != 'null')]
print('coupon not used: ', coupon_not_used.shape[0], '\n coupon used: ', coupon_used.shape[0], '\n no coupon: ', no_coupon.shape[0])

coupon not used:  977900 
 coupon used:  75382 
 no coupon:  701602


In [6]:
def time_minus():
    date = coupon_used['Date'].values
    date_received = coupon_used['Date_received'].values
    minus = []
    
    for i in tqdm(range(date.shape[0])):
        date_1 = datetime.datetime(int(date[i][ : 4]), int(date[i][4 : 6]), int(date[i][6 : 8]))
        date_2 = datetime.datetime(int(date_received[i][ : 4]), int(date_received[i][4 : 6]), int(date_received[i][6 : 8]))
        minus.append((date_1 - date_2).days)    

    coupon_used['days'] = minus


def label_appending():
    labels = []
    for day in tqdm(list(coupon_used['days'].values)):
        if day <= 15:
            labels.append(1)
        else:
            labels.append(0)

    coupon_used['labels'] = labels


def dr2rate(all_data):
    dr = all_data['Discount_rate'].values
    rate = []
    
    for discount_rate in tqdm(dr):
        if discount_rate == 'null':
            rate.append(0.0)
        elif discount_rate =='fixed':
            rate.append(0.2)
        else:
            try:
                rate.append(1 - (round(float(discount_rate.split(':')[1]) / float(discount_rate.split(':')[0]), 2)))
            except IndexError:
                rate.append(discount_rate)
    all_data['rate'] = rate

    
    
time_minus()
label_appending()

coupon_not_used['labels'] = 0
no_coupon['labels'] = 0

all_data = pd.concat([coupon_not_used, coupon_used, no_coupon])
dr2rate(all_data=all_data)

labels = all_data['labels'].values
all_data = all_data.drop(['Discount_rate', 'days', 'Date', 'Date_received', 'labels'], axis=1)

100%|████████████████████████████████████████████████████████████████████████| 75382/75382 [00:00<00:00, 211121.99it/s]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
100%|███████████████████████████████████████████████████████████████████████| 75382/75382 [00:00<00:00, 1180709.17it/s]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launc

In [7]:
all_data.head(10)

,Coupon_id,Distance,Merchant_id,User_id,rate
1,11002,1,4663,1439408,0.87
2,8591,0,2632,1439408,0.95
3,1078,0,2632,1439408,0.95
4,8591,0,2632,1439408,0.95
7,7610,0,3381,1832624,0.9
8,11951,1,3381,2029232,0.9
9,1532,0,450,2029232,0.83
10,12737,0,6459,2029232,0.95
13,1097,null,6901,2747744,0.8
15,10698,1,1579,196342,0.95


In [13]:
coupon_id = list(all_data['Coupon_id'].values)
# date = list(all_data['Date'].values)
# date_received = list(all_data['Date_received'].values)
distance = list(all_data['Distance'].values)

def fillna():  
    for i in range(len(coupon_id)):
        if coupon_id[i] == 'null':
            coupon_id[i] = -999
        else:
            pass
        
    for i in range(len(distance)):
        if distance[i] == 'null':
            distance[i] = np.random.randint(1, 10)
        else:
            pass

fillna()
all_data['Coupon_id'] = all_data['Coupon_id'].astype('int64')
# all_data['Date'] = all_data['Date'].astype('int64')
# all_data['Date_received'] = all_data['Date_received'].astype('int64')
all_data['Distance'] = all_data['Distance'].astype('int64')
all_data['rate'] = all_data['rate'].astype(float)

ValueError: invalid literal for int() with base 10: 'null'

In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1754884 entries, 1 to 1754881
Data columns (total 7 columns):
Coupon_id        int64
Date             int64
Date_received    int64
Distance         int64
Merchant_id      int64
User_id          int64
rate             float64
dtypes: float64(1), int64(6)
memory usage: 107.1 MB


In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20).split(all_data, labels)

for train_index, test_index in skf:
    X_train = all_data.iloc[train_index]
    y_train = labels[train_index]
    X_test = all_data.iloc[test_index]
    y_test = labels[test_index]
    break

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1403907, 7), (1403907,), (350977, 7), (350977,))

In [25]:
del offline_dataframe, coupon_not_used, coupon_used, no_coupon, all_data
gc.collect()

12783

In [30]:
X_train.head()

,Coupon_id,Date,Date_received,Distance,Merchant_id,User_id,rate
1,11002,-999,20160528,1,4663,1439408,0.87
2,8591,-999,20160217,0,2632,1439408,0.95
4,8591,-999,20160613,0,2632,1439408,0.95
8,11951,-999,20160129,1,3381,2029232,0.90
9,1532,-999,20160530,0,450,2029232,0.83


In [28]:
train = xgb.DMatrix(data=X_train, label=y_train, missing=-999)
val = xgb.DMatrix(data=X_test, label=y_test, missing=-999)

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 0.1,
    'max_depth': 5,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000, 
    'eval_metric': 'auc'
}
plst = params.items()
watch_list = [(train, 'train'), (val, 'val')]

In [29]:
model = xgb.train(plst, train, num_boost_round=100, evals=watch)

[0]	train-auc:0.998744	val-auc:0.998478
[1]	train-auc:0.998843	val-auc:0.998603
[2]	train-auc:0.998791	val-auc:0.99855
[3]	train-auc:0.999126	val-auc:0.999102
[4]	train-auc:0.999086	val-auc:0.99906
[5]	train-auc:0.999083	val-auc:0.999039
[6]	train-auc:0.999064	val-auc:0.999018
[7]	train-auc:0.99905	val-auc:0.999012
[8]	train-auc:0.999029	val-auc:0.998988
[9]	train-auc:0.999002	val-auc:0.998963
[10]	train-auc:0.999135	val-auc:0.999111
[11]	train-auc:0.999229	val-auc:0.999221
[12]	train-auc:0.9993	val-auc:0.999296
[13]	train-auc:0.999336	val-auc:0.999339
[14]	train-auc:0.99932	val-auc:0.999323
[15]	train-auc:0.999489	val-auc:0.999443
[16]	train-auc:0.999488	val-auc:0.999443
[17]	train-auc:0.99947	val-auc:0.999431
[18]	train-auc:0.999465	val-auc:0.999428
[19]	train-auc:0.999462	val-auc:0.999425
[20]	train-auc:0.999498	val-auc:0.999462
[21]	train-auc:0.999546	val-auc:0.999502
[22]	train-auc:0.999548	val-auc:0.9995
[23]	train-auc:0.999576	val-auc:0.999524
[24]	train-auc:0.99957	val-auc:0.99

KeyboardInterrupt: 

In [9]:
np.random.randint(1, 10)

1